In [1]:
import numpy as np
import math
import re
import itertools
import pandas as pd
import time
import os
import pickle

from Functions import *

In [56]:
# To merge .pk files

ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp']
full_dict={}
for j in range(1,5):
    base_dir='diff_'+str(j)
    diff_dict={}
    filenames = next(os.walk(base_dir), (None, None, []))[2]
    for file in filenames:
        full_dir=os.path.join(base_dir,file)
        with open(full_dir, 'rb') as handle:
            f1=pickle.load(handle)
        for i in list(f1):
            if i=='kwargs':
                del f1['kwargs']
                continue
            else:
                WA=assign_wells_mat(i)
                f1[i][1].update({'matrix':WA})
                mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, differentiate=j)
                n_wells=WA.shape[1]
                M_exp=np.round(mean_exp, 2)
                max_comp=np.max(np.sum(WA, axis=0))
                ls_met=[M_exp, max_comp, n_wells, perc_check,  extra_exp]
                f2=f1[i][0].drop(labels='matrix')
                idxs=list(f2.index)
                idxs+=['matrix']
                r_idxs=idxs[-1:]+idxs[:-1]
                dict_met={nm:list(f2[nm])+[val] for nm,val in zip(ls_names_met, ls_met)}

                f1[i][0]=pd.DataFrame(data=dict_met, index=idxs).reindex(r_idxs)

                
                f1[i][0]=f1[i][0].round({'mean_extra_exp':2})
            
        diff_dict.update(f1)
    full_str='Differentiate '+str(j)
    full_dict.update({full_str:diff_dict})

    
    


In [58]:
#output as pickle file
full_dir='Final_precomputed_file.pk'

#with open(full_dir, 'wb') as handle:
    #pickle.dump(full_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
# To merge files generated with inline_print
all_methods = ['matrix', 'multidim', 'random', 'STD', 'CT', 'Binary']
results = {}

for method_dir in all_methods:
    filenames = next(os.walk(method_dir), (None, None, []))[2]
    metrics_list = []
    for file in filenames:
        if file.endswith(".txt"):
            try:
                metrics = extract_metrics(file)
                metrics["method"] = method_dir
                metrics_list.append(metrics)
            except ValueError as e:
                print(e)
    
    if metrics_list:
        df = pd.DataFrame(metrics_list)
        df = df.sort_values(by="NS")
        
        # Group by diff value
        for diff_value, group_df in df.groupby("diff"):
            group_df = group_df.reset_index(drop=True)
            
            if diff_value not in results:
                results['Differentiate ' + str(diff_value)] = {}
                
            for ns_value, ns_group_df in group_df.groupby("NS"):
                ns_group_df = ns_group_df.drop(columns=["diff", "NS"]) 
                ns_group_df = ns_group_df.rename(columns={"NW": "n_wells", "MS": "max_compounds_per_well"})
               
                ns_group_df = ns_group_df.set_index(["method"])
                
                if ns_value not in results['Differentiate ' + str(diff_value)]:
                    results['Differentiate ' + str(diff_value)][ns_value] = [ns_group_df, {}]
                else:
                    existing_df, empty_dict = results['Differentiate ' + str(diff_value)][ns_value]
                    
                    merged_df = pd.merge(existing_df.reset_index(), ns_group_df.reset_index(),
                                         on=["method", "n_wells", "max_compounds_per_well"], how="outer", indicator=True)
                    
                    # Filter out rows that are duplicates (the '_merge' column indicates if it's from both dataframes)
                    new_entries_df = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=["_merge"]).set_index("method")
                    existing_entries_df = merged_df[merged_df['_merge'] == 'both'].drop(columns=["_merge"]).set_index("method")
                    
                    # Concatenate only new entries
                    combined_df = pd.concat([existing_entries_df, new_entries_df]).drop_duplicates().reset_index(drop=True)
                    
                    # Reassign the combined dataframe
                    results['Differentiate ' + str(diff_value)][ns_value] = [combined_df, {}]
            

In [5]:

# Output as pickle file
full_dir = 'Final_inline_precomputed_file.pk'

# First check if file already exists
# Merge new and old if it does
if os.path.exists(full_dir):
    with open(full_dir, "rb") as f:
        existing_results = pickle.load(f)
        
    for diff_value, ns_values in results.items():
        if diff_value not in existing_results:
            existing_df, existing_empty_dict = existing_results[diff_value][ns_value]
            
            for ns_value, (new_df, empty_dict) in ns_values.items():
                if ns_value in existing_results[diff_value]:
                    existing_df, existing_empty_dict = existing_results[diff_value][ns_value]
                
            
                    merged_df = pd.merge(existing_df.reset_index(), new_df.reset_index(),
                                         on=["method", "n_wells", "max_compounds_per_well"], how="outer", indicator=True)
                    
                    new_entries_df = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=["_merge"]).set_index("method")
                    existing_entries_df = merged_df[merged_df['_merge'] == 'both'].drop(columns=["_merge"]).set_index("method")
                    
                    combined_df = pd.concat([existing_entries_df, new_entries_df]).drop_duplicates().reset_index(drop=True)
                    existing_results[diff_value][ns_value] = [combined_df.set_index(["method"]), {}]
                else:
                    existing_results[diff_value][ns_value] = [new_df, empty_dict]
            
                
    merged_results = existing_results
    
else:
    merged_results = results

#with open(full_dir, 'wb') as handle:
    #pickle.dump(merged_results, handle, protocol=pickle.HIGHEST_PROTOCOL)